# Anomoly Detection with Time Series Exercies

## Exercises
file name: time_series_anomaly_detection.py or time_series_anomaly_detection.ipynb
- The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.

**Hint:** You will need to explore the database and significantly adjust your acquisition step (yay SQL!)

- Label students by the program they are in.
- Is it possible to identify the user_id of a staff member?
- Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.
- Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2.
- Plot the access activity of these students.

**BONUS:** Identify users who are viewing both the web dev and data science curriculum

In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env

In [2]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
sql = '''
    SELECT logs.date,
    logs.time,
    logs.user_id, 
    logs.path as endpoint, 
    logs.ip as source_ip,
    logs.cohort_id, 
    cohorts.name as cohort_name, 
    cohorts.start_date as cohort_start,
    cohorts.end_date as cohort_end, 
    cohorts.program_id
    FROM logs
    LEFT JOIN cohorts on cohorts.id=logs.cohort_id
    '''

In [3]:
import os
file = 'curriculum_logs.csv'
if os.path.isfile(file):
    df = pd.read_csv(file, index_col=0)
else:
    df = pd.read_sql(sql,url)
    df.to_csv(file)
df.head()

,date,user_id,endpoint,source_ip,cohort_id,cohort_name,cohort_start,cohort_end,program_id
0,2018-01-26,1,/,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,1,java-ii,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,1,java-ii/object-oriented-programming,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,1,slides/object_oriented_programming,97.105.19.61,8.0,Hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,2,javascript-i/conditionals,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0


In [4]:
df.shape

(900223, 9)

In [5]:
# check for dulicates
df.duplicated().sum()

347718

In [6]:
# look at duplicated rows
df[df.duplicated(keep=False)].head() #looks like entries are doubled, going to keep first only

,date,user_id,endpoint,source_ip,cohort_id,cohort_name,cohort_start,cohort_end,program_id
4,2018-01-26,2,javascript-i/conditionals,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0
5,2018-01-26,2,javascript-i/loops,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0
7,2018-01-26,3,javascript-i/functions,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0
8,2018-01-26,2,javascript-i/loops,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0
9,2018-01-26,4,javascript-i/functions,97.105.19.61,22.0,Teddy,2018-01-08,2018-05-17,2.0


In [7]:
# removing duplicate rows and show shape
df = df.drop_duplicates(keep='first')
df.shape

(552505, 9)

In [8]:
# replace blanks with
df = df.replace('', np.nan)

In [9]:
# check for nulls
df.isna().sum()

date                0
user_id             0
endpoint            1
source_ip           0
cohort_id       32122
cohort_name     32122
cohort_start    32122
cohort_end      32122
program_id      32122
dtype: int64

In [15]:
df[df.cohort_name=='Staff']

,date,user_id,endpoint,source_ip,cohort_id,cohort_name,cohort_start,cohort_end,program_id
166284,2018-12-06,53,appendix/further-reading/java/intellij-tomcat-...,97.105.19.58,28.0,Staff,2014-02-04,2014-02-04,2.0
166294,2018-12-06,314,javascript-i,97.105.19.58,28.0,Staff,2014-02-04,2014-02-04,2.0
166295,2018-12-06,314,javascript-i/coffee-project,97.105.19.58,28.0,Staff,2014-02-04,2014-02-04,2.0
166521,2018-12-06,40,spring,170.248.173.247,28.0,Staff,2014-02-04,2014-02-04,2.0
166522,2018-12-06,40,spring/fundamentals/security,170.248.173.247,28.0,Staff,2014-02-04,2014-02-04,2.0
...,...,...,...,...,...,...,...,...,...
900214,2021-04-21,64,javascript-i,71.150.217.33,28.0,Staff,2014-02-04,2014-02-04,2.0
900216,2021-04-21,64,jquery,71.150.217.33,28.0,Staff,2014-02-04,2014-02-04,2.0
900218,2021-04-21,64,jquery/personal-site,71.150.217.33,28.0,Staff,2014-02-04,2014-02-04,2.0
900219,2021-04-21,64,jquery/mapbox-api,71.150.217.33,28.0,Staff,2014-02-04,2014-02-04,2.0


In [14]:
df[df.isna()].cohort_name.unique()

array([nan], dtype=object)

In [ ]:
# drop nulls for now
df = df.dropna()
df.shape

In [ ]:
df.head()

## 1) Label Students by Program

In [ ]:
# how many program are there?
df.program_id.value_counts()

In [ ]:
# look at program 2 since it's the largest
list(df[df.program_id==2].cohort_name.unique())

In [ ]:
# look at program 2 since it's the largest
list(df[df.program_id==3].cohort_name.unique()) # looks like data science students

In [ ]:
# look at program 2 since it's the largest
list(df[df.program_id==1].cohort_name.unique())

In [ ]:
# look at program 2 since it's the largest
list(df[df.program_id==4].cohort_name.unique())

In [ ]:
# df subset by program_id, check paths to confirm its data science
df[df.program_id==3].endpoint.value_counts(normalize=True) # comfirms program_2 is data science

In [ ]:
# df subset by program_id, check path info for instruction type
df[df.program_id==2].endpoint.value_counts(normalize=True) # seems to be java related to 'web dev'

In [ ]:
# df subset by program_id, check path for instruction type
df[df.program_id==1].endpoint.value_counts(normalize=True) # seems to be java, html, nlp, dataset, mix of things. maybe for staff

In [ ]:
# investigate program_id 1 further
df[df.program_id==1].groupby('endpoint').date.count().sort_values(ascending=False).head(30) #mostly jaba so 'web dev'

In [ ]:
# df subset by program_id, check paths to confirm its data science
df[df.program_id==4].endpoint.value_counts(normalize=True) # seemto be html so 'web dev'

### program 3 is data science
### all others are web dev

In [ ]:
df['program_name'] = np.where(df.program_id==3,'data_science', 'web_dev')
df.head()

## 2) Is it possible to identify the user_id of a staff member?

In [ ]:
# find null for cohort_id

In [ ]:
df[df.cohort_name=='Staff'].groupby('user_id').count()

## 3) Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

In [ ]:
df.head()

In [ ]:
df.date = pd.to_datetime(df.date)
df.dtypes

In [ ]:
df = df.set_index(df.date)
df.head()

In [ ]:
df[df.index > df.cohort_end].user_id.unique()

## 4) Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2.

In [ ]:
pages = df['endpoint'].resample('d').count()
pages.head()

In [ ]:
pages.shape

In [ ]:
pages.plot()

In [ ]:
ema_7d = pages.ewm(span=7)
ema_7d

In [ ]:
# 7 day EMA
ema_7d = pages.ewm(span=7).mean()
ema_7d

In [ ]:
# the first ema is the first value of df so confirm by making them ==
ema_7d[0] == pages[0]

In [ ]:
# 14 day EMA
ema_14d = pages.ewm(span=14).mean()
ema_14d

In [ ]:
# 30 day EMA
ema_30d = pages.ewm(span=30).mean()
ema_30d

In [ ]:
# 90 day EMA
ema_90d = pages.ewm(span=90).mean()
ema_90d

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

ax.plot(pages.index, pages, label='Daily', alpha=.5)

ax.plot(pages.index, ema_7d, label='7 day EMA')
ax.plot(pages.index, ema_14d, label='14 day EMA')
ax.plot(pages.index, ema_30d, label='30 day EMA')
ax.plot(pages.index, ema_90d, label='60 day EMA')

In [ ]:
# create Bollinger Bandds midband

# set the window span
span = 30

# compute midband
midband = pages.ewm(span=span).mean()

midband.head()

In [ ]:
# compute exponential stdev
stdev = pages.ewm(span=span).std()
stdev.head()

In [ ]:
# compute upper and lower bands with K=2
ub = midband + stdev*2
lb = midband - stdev*2
ub, lb

In [ ]:
# concatenate ub and lb together into one df, bb
bb = pd.concat([ub, lb], axis=1)
bb.head()

In [ ]:
bb.columns = ['ub', 'lb']
bb.head()

In [ ]:
my_df = pd.concat([pages, midband, bb], axis=1)
my_df 

In [ ]:
my_df.columns = ['pages', 'midband', 'ub', 'lb']
my_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

ax.plot(my_df.index, my_df.pages, label='Number of Pages')

ax.plot(my_df.index, my_df.midband, label = '30-day EMA/midband')
ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
ax.plot(my_df.index, my_df.lb, label = 'Lower Band')

ax.legend(loc='best')
ax.set_ylabel('Number of pages')

plt.show()

In [ ]:
# create column %b for each pages values. No value for first observation
my_df['pct_b'] = (my_df['pages'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])

my_df.head()

In [ ]:
# find anomolies by filtering for %b above 1
my_df[my_df['pct_b']>1]

In [ ]:
# create df for one user
df.head()

In [ ]:
user = 1
span = 30

In [ ]:
# create df for single user
df = df[df.user_id == user]
df.head()

In [ ]:
# resample df.path daily and count to find how many hits for path
pages_one_user = df['endpoint'].resample('d').count()
pages_one_user.head()

In [ ]:
# Calculate upper and lower bollinger band and midband
weight = 2
midband = pages_one_user.ewm(span=span).mean()
stdev = pages_one_user.ewm(span=span).std()
ub = midband + stdev*weight
lb = midband - stdev*weight
ub, lb, midband

In [ ]:
# Add upper and lower band values to dataframe
bb = pd.concat([ub, lb], axis=1)
bb.head()

In [ ]:
# Combine all data into a single dataframe
my_df = pd.concat([pages_one_user, midband, bb], axis=1)
my_df.head()

In [ ]:
my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
my_df.head()

In [ ]:
# Calculate percent b and relevant user id to dataframe
my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
my_df.head()

In [ ]:
my_df['user_id'] = user
my_df.head()

In [ ]:
my_df[my_df.pct_b>1]

## Makes Functions

In [ ]:
def acquire_logs(user=env.user, password=env.password, host=env.host):
    '''
    This function uses env creditial to pull tables cohort and logs from database curriculum_logs.
    It read from a csv file if it exist or pull from the server and then write to a csv vile.
    '''
    
    # read from file if it exist 
    import os
    file = 'curriculum_logs.csv'
    if os.path.isfile(file):
        df = pd.read_csv(file, index_col=0)
    # or read from sql server and write to csv file
    else:
        url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
        sql = '''
        SELECT logs.date,
        logs.user_id, 
        logs.path as endpoint, 
        logs.ip as source_ip,
        logs.cohort_id, 
        cohorts.name as cohort_name, 
        cohorts.start_date as cohort_start,
        cohorts.end_date as cohort_end, 
        cohorts.program_id
        FROM logs
        LEFT JOIN cohorts on cohorts.id=logs.cohort_id
        '''
        
        # read from sql server and create dataframe
        df = pd.read_sql(sql,url)
        
#         # add column that names program_id == 3 as 'data_science' and others as 'web_dev'
#         df['program_name'] = np.where(df.program_id == '3', 'data_science', 'web_dev')
        
#         # add column that identified logs observations that were created after cohort grad date
#         df['accessed_after_grad'] = np.where(df.index > df.cohort_end, 'yes', 'no')        

        # write df to csv file
        df.to_csv(file)
    return df

def one_user_df_prep(df, user):
    '''
    This function returns a dataframe consisting of data for only a single defined user
    '''
    df = df[df.user_id == user]
    df.date = pd.to_datetime(df.date)
    df = df.set_index(df.date)
    pages_one_user = df['endpoint'].resample('d').count()
    return pages_one_user

def compute_pct_b(pages_one_user, span, weight, user):
    '''
    This function adds the %b of a bollinger band range for the page views of a single user's log activity
    '''
    # Calculate upper and lower bollinger band
    midband = pages_one_user.ewm(span=span).mean()
    stdev = pages_one_user.ewm(span=span).std()
    ub = midband + stdev*weight
    lb = midband - stdev*weight
    
    # Add upper and lower band values to dataframe
    bb = pd.concat([ub, lb], axis=1)
    
    # Combine all data into a single dataframe
    my_df = pd.concat([pages_one_user, midband, bb], axis=1)
    my_df.columns = ['pages_one_user', 'midband', 'ub', 'lb']
    
    # Calculate percent b and relevant user id to dataframe
    my_df['pct_b'] = (my_df['pages_one_user'] - my_df['lb'])/(my_df['ub'] - my_df['lb'])
    my_df['user_id'] = user
    return my_df

def plot_bands(my_df, user):
    '''
    This function plots the bolliger bands of the page views for a single user
    '''
    fig, ax = plt.subplots(figsize=(12,8))
    ax.plot(my_df.index, my_df.pages_one_user, label='Number of Pages, User: '+str(user))
    ax.plot(my_df.index, my_df.midband, label = 'EMA/midband')
    ax.plot(my_df.index, my_df.ub, label = 'Upper Band')
    ax.plot(my_df.index, my_df.lb, label = 'Lower Band')
    ax.legend(loc='best')
    ax.set_ylabel('Number of Pages')
    plt.show()

def find_anomalies(df, user, span, weight, plot=False):
    '''
    This function returns the records where a user's daily activity exceeded the upper limit of a bollinger band range
    '''
    
    # Reduce dataframe to represent a single user
    pages_one_user = one_user_df_prep(df, user)
    
    # Add bollinger band data to dataframe
    my_df = compute_pct_b(pages_one_user, span, weight, user)
    
    # Plot data if requested (plot=True)
    if plot:
        plot_bands(my_df, user)
    
    # Return only records that sit outside of bollinger band upper limit
    return my_df[my_df.pct_b>1]

In [ ]:
df = acquire_logs()
df.head()

In [ ]:
user = 1
span = 30
weight = 2
user_df = find_anomalies(df, user, span, weight)

anomalies = pd.DataFrame()
user_df = find_anomalies(df, user, span, weight)
anomalies = pd.concat([anomalies, user_df], axis=0)

In [ ]:
anomalies.head()

In [ ]:
span = 30
weight = 2

anomalies = pd.DataFrame()
for u in list(df.user_id.unique()):
    user_df = find_anomalies(df, u, span, weight)
    anomalies = pd.concat([anomalies, user_df], axis=0)

In [ ]:
anomalies.sort_values(by='pct_b', ascending=False)

In [ ]:
anomalies.sort_values(by='pages_one_user', ascending=False)

In [ ]:
anomalies.user_id.value_counts()

## 5) Plot the access activity of these students.
- Find iterative plotting technique

In [ ]:
# plot by page_one_user count
df_11 = one_user_df_prep(df, 11)
df_11.plot(figsize=(16,8))

In [ ]:
# plot by user_id count 
df_64 = one_user_df_prep(df, 64)
df_64.plot(figsize=(16,8))

## **BONUS:** Identify users who are viewing both the web dev and data science curriculum 

In [ ]:
df.head()

In [ ]:
# create column program_name that maps to program_id
df['program_name'] = np.where(df.program_id==3,'data_science', 'web_dev')
df.head()

In [ ]:
# find data science students that have logs for web deb pages
# subset df to data_science only
df_data_science = df[df.program_name=='data_science']
df_data_science.head()

In [ ]:
# find data science student endpoints that contain java or html
df_data_science.endpoint.str.contains(pat = 'html|java', case=False, regex=True).sum() # Too few to continue

In [ ]:
# make list of data science endpoints
ds_endpoints = df_data_science.endpoint.unique()
ds_endpoints = pd.Series(ds_endpoints)
ds_endpoints

In [ ]:
ds_endpoints.str.contains('java|html', case=False, regex=True)

In [ ]:
# find web dev students with endpoints of data science endpoints
df_web_dev = df[df.program_name=='web_dev']
df_web_dev.head()

In [ ]:
df_web_dev[df_web_dev.endpoint.isin(ds_endpoints)] # 234K observation of web dev have 